# Python で気軽に化学・化学工学
# 第 10 章 モデルを用いて y から x を推定する
## 10.2 仮想サンプルの予測および候補の選択

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

## 1. 回帰モデル (SVR モデル) の構築

In [ ]:
import pandas as pd # pandas のインポート
import numpy as np # NumPy のインポート

### 仮想的な樹脂材料のデータセット (virtual_resin.csv)

原料として 3 種類 (raw material 1, raw material 2, raw material 3) あり、それらの組成比と重合温度 (temperature)・重合時間 (time) をそれぞれ変えて樹脂材料が作られ、物性 a (property a) と物性 b (property b) が測定されたような 20 サンプルがあるとします。データセットのファイルは virtual_resin.csv です。

In [ ]:
dataset = pd.read_csv('virtual_resin.csv', index_col=0, header=0) # 仮想的な樹脂のデータセットの読み込み

In [ ]:
dataset # 念のため確認

In [ ]:
x = dataset.iloc[:, 2:] # 説明変数 x

In [ ]:
x # 念のための確認

In [ ]:
y = dataset.iloc[:, 0] # 目的変数 y (ここでは property_a のみ扱います)

In [ ]:
y # 念のため確認

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x = (x - x.mean()) / x.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_y = (y - y.mean()) / y.std() # トレーニングデータの目的変数の標準化

ガウシアンカーネルを用いた SVR

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
nonlinear_svr_cs = 2 ** np.arange(-5, 10, dtype=float) # 非線形SVR の C の候補

In [ ]:
nonlinear_svr_cs # 念のため確認

In [ ]:
nonlinear_svr_epsilons = 2 ** np.arange(-10, 0, dtype=float) # 非線形SVRのεの候補

In [ ]:
nonlinear_svr_epsilons # 念のため確認

In [ ]:
nonlinear_svr_gammas = 2 ** np.arange( -20, 10, dtype=float) # 非線形SVRのガウシアンカーネルのγの候補

In [ ]:
nonlinear_svr_gammas # 念のため確認

C・ε・γの最適化

In [ ]:
# 分散最大化によるガウシアンカーネルのγの最適化
variance_of_gram_matrix = []
autoscaled_x_array = np.array(autoscaled_x)
for nonlinear_svr_gamma in nonlinear_svr_gammas:
    gram_matrix = np.exp(- nonlinear_svr_gamma * ((autoscaled_x_array[:, np.newaxis] - autoscaled_x_array) ** 2).sum(axis=2))
    variance_of_gram_matrix.append(gram_matrix.var(ddof=1))

optimal_nonlinear_gamma = nonlinear_svr_gammas[np.where(variance_of_gram_matrix == np.max(variance_of_gram_matrix))[0][0]]

In [ ]:
optimal_nonlinear_gamma # 念のため確認

In [ ]:
fold_number = 10 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV # クロスバリデーションに使用
from sklearn.svm import SVR # SVR モデルの構築に使用

In [ ]:
cross_validation = KFold(n_splits=fold_number, random_state=9, shuffle=True) # クロスバリデーションの分割の設定

In [ ]:
# CV による ε の最適化
gs_cv = GridSearchCV(SVR(kernel='rbf', C=3, gamma=optimal_nonlinear_gamma),
                     {'epsilon': nonlinear_svr_epsilons},
                     cv=cross_validation)
gs_cv.fit(autoscaled_x, autoscaled_y)
optimal_nonlinear_epsilon = gs_cv.best_params_['epsilon']

In [ ]:
# CV による C の最適化
gs_cv = GridSearchCV(SVR(kernel='rbf', epsilon=optimal_nonlinear_epsilon, gamma=optimal_nonlinear_gamma),
                     {'C': nonlinear_svr_cs},
                     cv=cross_validation)
gs_cv.fit(autoscaled_x, autoscaled_y)
optimal_nonlinear_c = gs_cv.best_params_['C']

In [ ]:
# CV による γ の最適化
gs_cv = GridSearchCV(SVR(kernel='rbf', epsilon=optimal_nonlinear_epsilon, C=optimal_nonlinear_c),
                     {'gamma': nonlinear_svr_gammas},
                     cv=cross_validation)
gs_cv.fit(autoscaled_x, autoscaled_y)
optimal_nonlinear_gamma = gs_cv.best_params_['gamma']

In [ ]:
optimal_nonlinear_c # 念のため確認

In [ ]:
optimal_nonlinear_epsilon # 念のため確認

In [ ]:
optimal_nonlinear_gamma # 念のため確認

In [ ]:
model = SVR(kernel='rbf', C=optimal_nonlinear_c, epsilon=optimal_nonlinear_epsilon, gamma=optimal_nonlinear_gamma) # SVRモデルの宣言

In [ ]:
model.fit(autoscaled_x, autoscaled_y) # SVRモデル構築

トレーニングデータの y の値の推定

In [ ]:
estimated_y = pd.DataFrame(model.predict(autoscaled_x)) # pandas の DataFrame 型に変換

In [ ]:
estimated_y = estimated_y * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_y.index = x.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y # 念のため確認

In [ ]:
estimated_y.to_csv('estimated_y.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y, estimated_y.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y.max(), estimated_y.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y.min(), estimated_y.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y, estimated_y) # r2

In [ ]:
metrics.mean_absolute_error(y, estimated_y) # MAE

## 2. 構築されたモデルを用いた新たなサンプルの予測

y が未知のサンプルにおける、y の値の推定

In [ ]:
x_prediction = pd.read_csv('virtual_resin_x_for_prediction.csv', encoding='SHIFT-JIS', index_col=0)  # y が未知のデータセットの読み込み

In [ ]:
x_prediction  # 念のため確認

オートスケーリング

In [ ]:
autoscaled_x_prediction = (x_prediction - x.mean()) / x.std()

In [ ]:
estimated_y_prediction = pd.DataFrame(model.predict(autoscaled_x_prediction)) # 推定後、pandas の DataFrame 型に変換

In [ ]:
estimated_y_prediction = estimated_y_prediction * y.std() + y.mean() # スケールをもとに戻します

In [ ]:
estimated_y_prediction.index = x_prediction.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_prediction.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_prediction # 念のため確認

In [ ]:
estimated_y_prediction.to_csv('estimated_y_prediction.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

## 3. データ密度 (k-NN により計算) による AD の設定

In [ ]:
from sklearn.neighbors import NearestNeighbors # k-NN

In [ ]:
k_in_knn = 3

In [ ]:
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric='euclidean') # k-NN モデルを表す変数の作成

In [ ]:
ad_model.fit(autoscaled_x) # k-NN では、トレーニングデータの数値データを model_ad に格納することに対応します

In [ ]:
knn_distance_between_autoscaled_x, knn_index_autoscaled_x = ad_model.kneighbors(autoscaled_x, n_neighbors=k_in_knn+1)

In [ ]:
knn_distance_between_autoscaled_x # サンプルごとの k最近傍サンプルとの距離

In [ ]:
knn_index_autoscaled_x # サンプルごとの k最近傍サンプルのインデックス番号

In [ ]:
knn_distance_between_autoscaled_x = pd.DataFrame(knn_distance_between_autoscaled_x) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x.index = x.index # サンプル名をトレーニングデータのサンプル名にします

In [ ]:
knn_distance_between_autoscaled_x # 念のため確認

In [ ]:
mean_of_knn_distance_between_autoscaled_x = knn_distance_between_autoscaled_x.iloc[:, 1:].mean(axis=1) # 自分以外の k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x # 念のため確認

次のセルで距離の平均の小さい順に並び替えます。なお、ascending=False とすると、大きい順に並び替えることができます。

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x = mean_of_knn_distance_between_autoscaled_x.sort_values(ascending=True)

In [ ]:
sorted_mean_of_knn_distance_between_autoscaled_x # 念のため確認

In [ ]:
alpha = 0.95 # この値を大きくすると、AD 内のサンプルが多くなります

In [ ]:
round(autoscaled_x.shape[0] * alpha) # 距離の小さい α ×100 % のサンプル数

In [ ]:
ad_threshold = sorted_mean_of_knn_distance_between_autoscaled_x.iloc[round(autoscaled_x.shape[0] * alpha) - 1] # トレーニング化合物の alpha % が含まれるようにしきい値を設定

In [ ]:
ad_threshold

#### テストデータに対して、AD の中か外かを判定

In [ ]:
knn_distance_between_autoscaled_x_prediction, knn_index_autoscaled_x_prediction = ad_model.kneighbors(autoscaled_x_prediction)

In [ ]:
knn_distance_between_autoscaled_x_prediction = pd.DataFrame(knn_distance_between_autoscaled_x_prediction) # DataFrame型に変換

In [ ]:
knn_distance_between_autoscaled_x_prediction.index = x_prediction.index # サンプル名をテストデータのサンプル名にします

In [ ]:
mean_of_knn_distance_between_autoscaled_x_prediction = knn_distance_between_autoscaled_x_prediction.mean(axis=1) # k_in_knn 個の距離の平均

In [ ]:
mean_of_knn_distance_between_autoscaled_x_prediction # 念のため確認

In [ ]:
estimated_y_prediction_inside_ad = estimated_y_prediction.loc[mean_of_knn_distance_between_autoscaled_x_prediction <= ad_threshold] # AD 内のサンプルにおける、y の予測値

In [ ]:
estimated_y_prediction_inside_ad.shape # AD 内のサンプル数を確認

In [ ]:
estimated_y_prediction_inside_ad.to_csv('estimated_y_prediction_inside_ad.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

サンプルごとの、y の予測値と k-NN 距離を保存

In [ ]:
mean_of_knn_distance_between_autoscaled_x_prediction = pd.DataFrame(mean_of_knn_distance_between_autoscaled_x_prediction, index=x_prediction.index, columns=['knn distance'])

In [ ]:
mean_of_knn_distance_between_autoscaled_x_prediction # 念のため確認

In [ ]:
results_prediction = pd.concat([estimated_y_prediction, mean_of_knn_distance_between_autoscaled_x_prediction], axis=1)

In [ ]:
results_prediction # 念のため確認

In [ ]:
results_prediction.to_csv('results_prediction.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください